In [ ]:
# default_exp agricultural_pricing_analytics

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

In [1]:
#export
import pandas as pd
from fastcore.script import *
from datetime import datetime
import numpy as np
import plotly.express as px
from ipywidgets import widgets


### Load the data

In [2]:
def load_data(path):
    df = pd.read_csv(path,header=0,
                     parse_dates = ['date'], 
                     date_parser= lambda x: datetime.strptime(x, "%B_%d_%Y"),
                     delimiter="|")
    
    print(df.info())
    return df
    

In [3]:
#load the retail data
path = "retail/data/output/september_04_2021/retail_prices_september_04_2021.csv"
retail_df = load_data(path)
retail_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1024 non-null   datetime64[ns]
 1   commodity       1008 non-null   object        
 2   variety-source  1008 non-null   object        
 3   region          1024 non-null   object        
 4   store           1024 non-null   object        
 5   price           1008 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 48.1+ KB
None


,date,commodity,variety-source,region,store,price
0,2021-09-04,Broccoli,Imported,cross roads,sampars,2416.67
1,2021-09-04,Broccoli,Imported,cross roads,hi-lo,2332
2,2021-09-04,Broccoli,Imported,waterloo road,mega mart,2140
3,2021-09-04,Broccoli,Imported,new kingston,John R Wong,1832
4,2021-09-04,Broccoli,Imported,halfway tree,Brooklyn,1924.98


In [4]:
# load urban municipal markets data
path = "urban_municipal/data/output/september_04_2021/urban_municipal_market_prices_september_04_2021.csv"
urban_municipal_markets_df = pd.read_csv(path,
                                         parse_dates = ['date'], 
                                         date_parser= lambda x: datetime.strptime(x, "%B_%d_%Y"),
                                         names= ["date","market","commodity","variety source","low","high","most frequent"],
                                         delimiter="|")
urban_municipal_markets_df.head()

,date,market,commodity,variety source,low,high,most frequent
0,2021-09-04,papine,Broccoli,Local,1760.0,1760.0,1760.0
1,2021-09-04,papine,Cabbage (Green),Local,550.0,550.0,550.0
2,2021-09-04,papine,Cabbage (Red),Local,NaN,NaN,NaN
3,2021-09-04,papine,Callaloo,Shredded,440.0,440.0,440.0
4,2021-09-04,papine,Callaloo,Whole,220.0,330.0,330.0


In [9]:
!ls wholesale/data/output/september_04_2021/

In [5]:
# load kingston supermarket prices data
path = "wholesale/data/output/september_04_2021/kingston_supermarket_prices_september_04_2021.csv"
# kingston_supermarket_prices_df = load_data(path)
kingston_supermarket_prices_df = pd.read_csv(path,delimiter="|")
kingston_supermarket_prices_df["date"] = "2021-09-04"
kingston_supermarket_prices_df['date'] = pd.to_datetime(kingston_supermarket_prices_df['date'])
kingston_supermarket_prices_df.head()
kingston_supermarket_prices_df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'wholesale/data/output/september_04_2021/kingston_supermarket_prices_september_04_2021.csv'

In [10]:
# load farmgate price data
path = "farmgate/data/output/out.csv"
farm_gate_prices_df = pd.read_csv(path,delimiter="|")
farm_gate_prices_df = farm_gate_prices_df[farm_gate_prices_df["date"] != "date"]
farm_gate_prices_df['date'] = pd.to_datetime(farm_gate_prices_df['date'], format="%B_%d_%Y")
farm_gate_prices_df["high"] = pd.to_numeric(farm_gate_prices_df["high"], errors='coerce')
farm_gate_prices_df["low"] = pd.to_numeric(farm_gate_prices_df["low"], errors='coerce')
farm_gate_prices_df["most frequent"] = pd.to_numeric(farm_gate_prices_df["most frequent"], errors='coerce')
df =farm_gate_prices_df.copy()


FileNotFoundError: [Errno 2] No such file or directory: 'farmgate/data/output/out.csv'

In [11]:
df.head()

NameError: name 'df' is not defined

In [81]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

In [82]:
df.sample(3)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
248677,2013,6,28,NaN,NaN,NaN,NaN,EV,N17185,4111,EWR,CHS,NaN,628,NaN,NaN
1223,2013,1,2,1220.0,-9.0,1513.0,-13.0,B6,N203JB,27,JFK,TPA,161.0,1005,12.0,20.0
105576,2013,12,25,1411.0,12.0,1656.0,-18.0,B6,N712JB,213,JFK,LGB,326.0,2465,14.0,11.0


In [91]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)

In [92]:
# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

In [93]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


In [94]:
origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [95]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])

In [98]:
# load farmgate price data
path = "farmgate/data/output/out.csv"
farm_gate_prices_df = pd.read_csv(path,delimiter="|")
farm_gate_prices_df = farm_gate_prices_df[farm_gate_prices_df["date"] != "date"]
farm_gate_prices_df['date'] = pd.to_datetime(farm_gate_prices_df['date'], format="%B_%d_%Y")
farm_gate_prices_df["high"] = pd.to_numeric(farm_gate_prices_df["high"], errors='coerce')
farm_gate_prices_df["low"] = pd.to_numeric(farm_gate_prices_df["low"], errors='coerce')
farm_gate_prices_df["most frequent"] = pd.to_numeric(farm_gate_prices_df["most frequent"], errors='coerce')
df =farm_gate_prices_df.copy()
df.head()

,date,parish,commodity,variety source,low,high,most frequent,supply,grade
0,2021-04-03,Manchester,broccoli,local,770.0,880.0,825.0,scarce,excellent
1,2021-04-03,Manchester,cabbage (green),local,44.0,88.0,60.5,moderate,excellent
2,2021-04-03,Manchester,cabbage (red),local,NaN,NaN,NaN,-,-
3,2021-04-03,Manchester,callaloo,shredded,NaN,NaN,NaN,-,-
4,2021-04-03,Manchester,callaloo,whole,77.0,110.0,88.0,moderate,excellent


In [104]:
commodity = widgets.Dropdown(
    options=list(df['commodity'].unique()),
    value='broccoli',
    description='Commodity:',
)

container = widgets.HBox(children=[commodity])


In [109]:
def validate():
    return True
   


def response(change):
    if validate():
        if commodity.value:
            temp_df = df[df.commodity == commodity.value]
            x1 = temp_df['supply']
        
            with g.batch_update():
                 g.data[0].x = x1
            
            

In [112]:
commodity.observe(response, names="value")

In [127]:
trace1 = go.Histogram(x=df, opacity=1, name='Supply')

g = go.FigureWidget(data=[trace1],
                    layout=go.Layout(
                        title=dict(
                            text='Farm Gate Prices'
                        ),
                        barmode='overlay'
                    ))

In [126]:
# container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,g])

In [66]:
## What is the availability of different produces
df = df.dropna()

fig = px.scatter(df, x="date", 
                 y="low",
                 title='Price', 
                 color='supply',
                 size="high",
                 hover_data=['parish','variety source','high','low','supply'],
                 facet_col_wrap=3,
                 
                )
fig.show()

In [58]:
fig = px.bar(df, x='date', y='supply')
fig.show()

In [55]:
df.head()

,date,parish,commodity,variety source,low,high,most frequent,supply,grade
0,2021-04-03,Manchester,broccoli,local,770.0,880.0,825.0,scarce,excellent
1,2021-04-03,Manchester,cabbage (green),local,44.0,88.0,60.5,moderate,excellent
2,2021-04-03,Manchester,cabbage (red),local,NaN,NaN,NaN,-,-
3,2021-04-03,Manchester,callaloo,shredded,NaN,NaN,NaN,-,-
4,2021-04-03,Manchester,callaloo,whole,77.0,110.0,88.0,moderate,excellent


In [61]:
df_yam = df.query("commodity == 'yam' and parish in ['Manchester','St. Andrew','St. Catherine','Clarendon','Portland', 'St. Elizabeth','Westmoreland','Hanover','St. James','Trelawny','St. Ann','St. Mary','Portland','St. Thomas']" )
df_yam = df_yam.dropna()

fig = px.scatter(df_yam, x="date", 
                 y="low",
                 title='Price', 
                 color='variety source',
                 size="high",
                 hover_data=['parish','variety source','high','low','supply'],
                 facet_col_wrap=3,
                 
                )
fig.update_xaxes(rangeslider_visible=True)

fig.show()

In [ ]:
fig = px.histogram(df_yam, x="date", 
                 y="high",
                 title='Price', 
                 color='variety source',
                 hover_data=['parish','variety source','high','low','supply'],
                 facet_col_wrap=3,
                 histfunc="max",
                 facet_col='parish' 
                )

fig.show()


In [ ]:
import plotly.express as px

fig = px.box(df_yam, x = "high")
fig.show()


In [ ]:
fig = px.strip(df_yam, x='date', y='high')
fig.show()

In [ ]:
fig = px.histogram(df_yam, x="low")
fig.show()

In [ ]:
fig = px.density_heatmap(df_yam, x="date", y="high")
fig.show()